In [1]:
#use CSV logger to records logs from fit function
#CSV logger is part of callbacks so you must call it in the fit function as a callback
#site: https://keras.io/api/callbacks/csv_logger/

import numpy as np #for statistics and linear algebra computation
import pandas as pd #for data processing and reading in data

from scipy import stats #statistical tools
from sklearn.model_selection import train_test_split #splitting data into training and test sets

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report #for computing confusion matrix

#import keras library to implement neural networks
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#import optimizers to train data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import binary_crossentropy

In [2]:
data = pd.read_csv("dataset.csv")#read dataset into variable

x = data.drop('CustomerId', axis=1)
x = x.drop('HasCrCard', axis=1)
x = x.drop('EstimatedSalary', axis=1)
x = x.drop('Surname', axis=1)
x = x.drop('Balance', axis=1)
x = x.drop('Exited', axis=1) #drop output variable you want to find from x data

#convert geography values to int values
#1 = Spain, 2 = France, 3 = Germany
x['Geography'] = x['Geography'].replace(['Spain', 'France', 'Germany'], [1, 2, 3]) #replace a value with a new value
        
#convert gender values to int values
#Female = 0, Male = 1
x['Gender'] = x['Gender'].replace(['Female', 'Male'], [0, 1])

# x #get input values that have a correlation with target output 'Exited'

y = data['Exited']
# y #separate output values from dataset

In [3]:
#split dataset into training and testing set (80-20 split)
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2, random_state=34) #set random state to an integer for identical splits

#need to scale data so that there aren't any outrageous values or distortions because of varying ranges of values
scale = MinMaxScaler()
#scale training samples
xTrain_scaled = scale.fit_transform(xTrain)
xTest_scaled = scale.fit_transform(xTest)

In [4]:
#design F1-score function 
#F1-score = 2pr/(p+r)
#https://keras.io/guides/writing_your_own_callbacks/, reference site
class callbackValues(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None): #epoch level method called at end of each epoch
        keys = list(logs.keys()) #obtain what metrics were calculated and obtained each epoch
        #print("Info obtained at end of epoch ", keys)
        #here calculate the f1-score and print it out
        train_precision = logs[keys[2]]
        test_precision = logs[keys[6]]
        train_recall = logs[keys[3]]
        test_recall = logs[keys[7]]
        train_denom = train_precision + train_recall #calculate f1 score for training data at each epoch
        test_denom = test_precision + test_recall #calculate f1 score for test data at each epoch
        #below is an error check to prevent division by zero
        if (train_denom <= 0.0):
            train_denom = 1
        if (test_denom <= 0.0):
            test_denom = 1
        
        train_f1 = 2 * train_precision * train_recall / (train_denom)
        test_f1 = 2 * test_precision * test_recall / (test_denom)
        print("Training F1 score: ", train_f1)
        print("Validation F1 score: ", test_f1)

In [5]:
#although ANN 2 was the best in terms of accuracy, I decided to use ANN 3 (second best) because it 
    #obtains close to the accuracy obtained in ANN 2 in less time
ANN3 = keras.Sequential([
    keras.Input(shape=7), #add input layer here
    layers.Dense(15, name="layer1"), #at this point it should have the weighted sum of the values from the input layer
    layers.Dense(20, activation="relu", name="layer2"), #apply activation function
    layers.Dense(10, activation="relu", name="layer3"),
    layers.Dense(5, activation="relu", name="layer4"),
    layers.Dense(2, activation="relu", name="layer5"),
    layers.Dense(1, activation="sigmoid", name="layer6")
])

In [6]:
#train and validate model for ANN 3
metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
ANN3.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=metrics)
ANN3.fit(x=xTrain_scaled, y=yTrain, validation_data=(xTest_scaled, yTest), batch_size=150, epochs=1100, shuffle=False,verbose=2, callbacks=[callbackValues()])

Epoch 1/1100
48/48 - 5s - loss: 0.6916 - accuracy: 0.4418 - precision: 0.1643 - recall: 0.4175 - val_loss: 0.6869 - val_accuracy: 0.6278 - val_precision: 0.1416 - val_recall: 0.1850
Training F1 score:  0.23578627775947847
Validation F1 score:  0.1604010006133059
Epoch 2/1100
48/48 - 0s - loss: 0.6823 - accuracy: 0.7343 - precision: 0.1285 - recall: 0.0498 - val_loss: 0.6754 - val_accuracy: 0.8083 - val_precision: 1.0000 - val_recall: 0.0029
Training F1 score:  0.0718098026961648
Validation F1 score:  0.005763688632349697
Epoch 3/1100
48/48 - 0s - loss: 0.6702 - accuracy: 0.7937 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.6611 - val_accuracy: 0.8078 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Training F1 score:  0.0
Validation F1 score:  0.0
Epoch 4/1100
48/48 - 0s - loss: 0.6542 - accuracy: 0.7937 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.6419 - val_accuracy: 0.8078 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Training F1 score:  0.0
Val

Training F1 score:  0.0
Validation F1 score:  0.0
Epoch 35/1100
48/48 - 0s - loss: 0.4816 - accuracy: 0.7937 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.4641 - val_accuracy: 0.8078 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Training F1 score:  0.0
Validation F1 score:  0.0
Epoch 36/1100
48/48 - 0s - loss: 0.4802 - accuracy: 0.7937 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.4629 - val_accuracy: 0.8078 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Training F1 score:  0.0
Validation F1 score:  0.0
Epoch 37/1100
48/48 - 0s - loss: 0.4789 - accuracy: 0.7937 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.4617 - val_accuracy: 0.8078 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Training F1 score:  0.0
Validation F1 score:  0.0
Epoch 38/1100
48/48 - 0s - loss: 0.4776 - accuracy: 0.7937 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.4605 - val_accuracy: 0.8078 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Trai

48/48 - 0s - loss: 0.4457 - accuracy: 0.8107 - precision: 0.7402 - recall: 0.1266 - val_loss: 0.4327 - val_accuracy: 0.8239 - val_precision: 0.6593 - val_recall: 0.1734
Training F1 score:  0.21621621660671547
Validation F1 score:  0.2745995392353998
Epoch 68/1100
48/48 - 0s - loss: 0.4448 - accuracy: 0.8111 - precision: 0.7273 - recall: 0.1347 - val_loss: 0.4320 - val_accuracy: 0.8256 - val_precision: 0.6667 - val_recall: 0.1850
Training F1 score:  0.2272727316885721
Validation F1 score:  0.28959275698847775
Epoch 69/1100
48/48 - 0s - loss: 0.4440 - accuracy: 0.8115 - precision: 0.7192 - recall: 0.1414 - val_loss: 0.4314 - val_accuracy: 0.8244 - val_precision: 0.6500 - val_recall: 0.1879
Training F1 score:  0.23635339653762283
Validation F1 score:  0.2914798266285395
Epoch 70/1100
48/48 - 0s - loss: 0.4432 - accuracy: 0.8114 - precision: 0.7029 - recall: 0.1481 - val_loss: 0.4308 - val_accuracy: 0.8244 - val_precision: 0.6442 - val_recall: 0.1936
Training F1 score:  0.24471635157635854

Training F1 score:  0.3247082670593063
Validation F1 score:  0.3695238011539657
Epoch 99/1100
48/48 - 0s - loss: 0.4312 - accuracy: 0.8153 - precision: 0.6585 - recall: 0.2168 - val_loss: 0.4227 - val_accuracy: 0.8167 - val_precision: 0.5444 - val_recall: 0.2832
Training F1 score:  0.3262411425714146
Validation F1 score:  0.372623561597773
Epoch 100/1100
48/48 - 0s - loss: 0.4309 - accuracy: 0.8151 - precision: 0.6571 - recall: 0.2168 - val_loss: 0.4225 - val_accuracy: 0.8172 - val_precision: 0.5470 - val_recall: 0.2861
Training F1 score:  0.3260759576920649
Validation F1 score:  0.37571158114633296
Epoch 101/1100
48/48 - 0s - loss: 0.4307 - accuracy: 0.8153 - precision: 0.6578 - recall: 0.2175 - val_loss: 0.4224 - val_accuracy: 0.8178 - val_precision: 0.5495 - val_recall: 0.2890
Training F1 score:  0.32692308080531773
Validation F1 score:  0.37878789241905464
Epoch 102/1100
48/48 - 0s - loss: 0.4305 - accuracy: 0.8151 - precision: 0.6565 - recall: 0.2175 - val_loss: 0.4222 - val_accur

Training F1 score:  0.36444886605487176
Validation F1 score:  0.39697540955770466
Epoch 130/1100
48/48 - 0s - loss: 0.4244 - accuracy: 0.8222 - precision: 0.6930 - recall: 0.2478 - val_loss: 0.4161 - val_accuracy: 0.8222 - val_precision: 0.5707 - val_recall: 0.3035
Training F1 score:  0.3650793706985162
Validation F1 score:  0.39622640926076547
Epoch 131/1100
48/48 - 0s - loss: 0.4242 - accuracy: 0.8222 - precision: 0.6930 - recall: 0.2478 - val_loss: 0.4159 - val_accuracy: 0.8222 - val_precision: 0.5707 - val_recall: 0.3035
Training F1 score:  0.3650793706985162
Validation F1 score:  0.39622640926076547
Epoch 132/1100
48/48 - 0s - loss: 0.4240 - accuracy: 0.8224 - precision: 0.6936 - recall: 0.2485 - val_loss: 0.4158 - val_accuracy: 0.8211 - val_precision: 0.5645 - val_recall: 0.3035
Training F1 score:  0.365889933871882
Validation F1 score:  0.39473683274538957
Epoch 133/1100
48/48 - 0s - loss: 0.4239 - accuracy: 0.8219 - precision: 0.6897 - recall: 0.2485 - val_loss: 0.4156 - val_ac

...

48/48 - 0s - loss: 0.3480 - accuracy: 0.8575 - precision: 0.7476 - recall: 0.4667 - val_loss: 0.3436 - val_accuracy: 0.8606 - val_precision: 0.6834 - val_recall: 0.5116
Training F1 score:  0.5746268741869466
Validation F1 score:  0.5851239668949236
Epoch 947/1100
48/48 - 0s - loss: 0.3480 - accuracy: 0.8575 - precision: 0.7476 - recall: 0.4667 - val_loss: 0.3436 - val_accuracy: 0.8606 - val_precision: 0.6834 - val_recall: 0.5116
Training F1 score:  0.5746268741869466
Validation F1 score:  0.5851239668949236
Epoch 948/1100
48/48 - 0s - loss: 0.3479 - accuracy: 0.8576 - precision: 0.7478 - recall: 0.4673 - val_loss: 0.3436 - val_accuracy: 0.8606 - val_precision: 0.6834 - val_recall: 0.5116
Training F1 score:  0.5752175788914894
Validation F1 score:  0.5851239668949236
Epoch 949/1100
48/48 - 0s - loss: 0.3479 - accuracy: 0.8576 - precision: 0.7478 - recall: 0.4673 - val_loss: 0.3436 - val_accuracy: 0.8606 - val_precision: 0.6834 - val_recall: 0.5116
Training F1 score:  0.5752175788914894


Training F1 score:  0.5763975235420743
Validation F1 score:  0.5804311655457957
Epoch 978/1100
48/48 - 0s - loss: 0.3476 - accuracy: 0.8579 - precision: 0.7484 - recall: 0.4687 - val_loss: 0.3435 - val_accuracy: 0.8600 - val_precision: 0.6822 - val_recall: 0.5087
Training F1 score:  0.5763975235420743
Validation F1 score:  0.5827814594396739
Epoch 979/1100
48/48 - 0s - loss: 0.3476 - accuracy: 0.8579 - precision: 0.7484 - recall: 0.4687 - val_loss: 0.3435 - val_accuracy: 0.8600 - val_precision: 0.6822 - val_recall: 0.5087
Training F1 score:  0.5763975235420743
Validation F1 score:  0.5827814594396739
Epoch 980/1100
48/48 - 0s - loss: 0.3476 - accuracy: 0.8578 - precision: 0.7481 - recall: 0.4680 - val_loss: 0.3435 - val_accuracy: 0.8600 - val_precision: 0.6822 - val_recall: 0.5087
Training F1 score:  0.5758077861573623
Validation F1 score:  0.5827814594396739
Epoch 981/1100
48/48 - 0s - loss: 0.3476 - accuracy: 0.8578 - precision: 0.7481 - recall: 0.4680 - val_loss: 0.3436 - val_accura

48/48 - 0s - loss: 0.3473 - accuracy: 0.8581 - precision: 0.7497 - recall: 0.4680 - val_loss: 0.3437 - val_accuracy: 0.8617 - val_precision: 0.6887 - val_recall: 0.5116
Training F1 score:  0.576285231439407
Validation F1 score:  0.5870646598939315
Epoch 1010/1100
48/48 - 0s - loss: 0.3473 - accuracy: 0.8582 - precision: 0.7505 - recall: 0.4680 - val_loss: 0.3437 - val_accuracy: 0.8617 - val_precision: 0.6887 - val_recall: 0.5116
Training F1 score:  0.5765242623052002
Validation F1 score:  0.5870646598939315
Epoch 1011/1100
48/48 - 0s - loss: 0.3472 - accuracy: 0.8583 - precision: 0.7508 - recall: 0.4687 - val_loss: 0.3437 - val_accuracy: 0.8617 - val_precision: 0.6887 - val_recall: 0.5116
Training F1 score:  0.5771144389704543
Validation F1 score:  0.5870646598939315
Epoch 1012/1100
48/48 - 0s - loss: 0.3472 - accuracy: 0.8583 - precision: 0.7508 - recall: 0.4687 - val_loss: 0.3438 - val_accuracy: 0.8617 - val_precision: 0.6887 - val_recall: 0.5116
Training F1 score:  0.577114438970454

48/48 - 0s - loss: 0.3467 - accuracy: 0.8587 - precision: 0.7500 - recall: 0.4727 - val_loss: 0.3442 - val_accuracy: 0.8622 - val_precision: 0.6842 - val_recall: 0.5260
Training F1 score:  0.5799256477034578
Validation F1 score:  0.594771262869545
Epoch 1041/1100
48/48 - 0s - loss: 0.3467 - accuracy: 0.8589 - precision: 0.7503 - recall: 0.4734 - val_loss: 0.3443 - val_accuracy: 0.8622 - val_precision: 0.6842 - val_recall: 0.5260
Training F1 score:  0.5805119746394737
Validation F1 score:  0.594771262869545
Epoch 1042/1100
48/48 - 0s - loss: 0.3467 - accuracy: 0.8587 - precision: 0.7500 - recall: 0.4727 - val_loss: 0.3442 - val_accuracy: 0.8622 - val_precision: 0.6842 - val_recall: 0.5260
Training F1 score:  0.5799256477034578
Validation F1 score:  0.594771262869545
Epoch 1043/1100
48/48 - 0s - loss: 0.3467 - accuracy: 0.8593 - precision: 0.7527 - recall: 0.4734 - val_loss: 0.3442 - val_accuracy: 0.8617 - val_precision: 0.6830 - val_recall: 0.5231
Training F1 score:  0.5812319225192434


48/48 - 0s - loss: 0.3463 - accuracy: 0.8589 - precision: 0.7519 - recall: 0.4714 - val_loss: 0.3442 - val_accuracy: 0.8628 - val_precision: 0.6868 - val_recall: 0.5260
Training F1 score:  0.5794701971525242
Validation F1 score:  0.5957446898409112
Epoch 1072/1100
48/48 - 0s - loss: 0.3463 - accuracy: 0.8590 - precision: 0.7527 - recall: 0.4714 - val_loss: 0.3442 - val_accuracy: 0.8628 - val_precision: 0.6868 - val_recall: 0.5260
Training F1 score:  0.579710144774238
Validation F1 score:  0.5957446898409112
Epoch 1073/1100
48/48 - 0s - loss: 0.3463 - accuracy: 0.8589 - precision: 0.7519 - recall: 0.4714 - val_loss: 0.3442 - val_accuracy: 0.8628 - val_precision: 0.6868 - val_recall: 0.5260
Training F1 score:  0.5794701971525242
Validation F1 score:  0.5957446898409112
Epoch 1074/1100
48/48 - 0s - loss: 0.3463 - accuracy: 0.8589 - precision: 0.7519 - recall: 0.4714 - val_loss: 0.3442 - val_accuracy: 0.8622 - val_precision: 0.6842 - val_recall: 0.5260
Training F1 score:  0.579470197152524

In [7]:
#predict output values from judge.csv file and output predictions to a file named judge-pred.csv with the customerID and Exited value
judge = pd.read_csv("judge.csv")

judgeID = judge['CustomerId']
# print(judgeID[0:10])
judge = judge.drop('CustomerId', axis=1)
judge = judge.drop('HasCrCard', axis=1)
judge = judge.drop('EstimatedSalary', axis=1)
judge = judge.drop('Surname', axis=1)
judge = judge.drop('Balance', axis=1)

judge['Geography'] = judge['Geography'].replace(['Spain', 'France', 'Germany'], [1, 2, 3]) 
judge['Gender'] = judge['Gender'].replace(['Female', 'Male'], [0, 1])

#scale judge data
judge_scaled = scale.fit_transform(judge)

# csv_logger = keras.callbacks.CSVLogger('judge-pred.csv')
yPredict = (ANN3.predict(judge_scaled) > 0.5).astype("int32")
# yPredict = ANN3.predict(judge_scaled) #gets predictions in probabilities
yPredict

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
    

In [8]:
#save predictions to file
outfile = open('judge-pred.csv', 'w')
outfile.write("CustomerID, Exited\n")
for i in range(len(yPredict)):
    outfile.write(str(judgeID[i]))
    outfile.write(", ")
    outfile.write(str(yPredict[i][0]))
    outfile.write("\n")
outfile.close()